## Background
Operators offer capping products which limit the amount their customers are charged over a certain period (e.g. daily or weekly). For example:

For a daily capping limit of 5 dollars:

| Day | Amount | Adjusted Amount | Accumulated for the day |
|:----|:-------|:-|:-|
| Trip 1 day 1 | 2 | 2 | 2 |
| Trip 2 day 1 | 2 | 2| 4 | 
| Trip 3 day 1| 2 | 1 | 5 | 
| Trip 1 day 2 | 3 | 3 | 3 | 
| Trip 2 day 2 | 3 | 2 | 5 | 
| Trip 3 day 2 | 3 | 0 | 5 | 

 
## Given:
Given the data dictionary below

#### Products
This table contains the capping product definitions

| Field                              | Description |
|:-----------------------------------|:--|
| id                                 | Unique ID |  
| product_type                       | CAPPING |  
| capping_type                       | DAILY or WEEKLY |  
| created_date                       | Date created |  
| start_date_utc                     | Effective start date |  
| status                             | ACTIVE or INACTIVE |  
| rules_default_limit                | Capping limit amount |  
| rules_max_fare_value               | Fare above this are not included in the capping |  
| rules_transaction_types            | Type of fare |  
| rules_from_time_local              | Time of day in which capping starts effectivity |  
| rules_to_time_local                | Time of day in which capping ends effectivity |  
| rules_weekly_cap_start_day         | If weekly capping, this is the start day (e.g. SUNDAY, MONDAY) |  
| rules_capping_duration_in_minutes  | If product_type is time based, free transfer window |  
| rules_number_of_transfer           | If product_type is time based, number of free transfer |  

#### Trips
This table contains the trips

| Field                       | Description | 
|:----------------------------|:--|
| charge_id                   | Unique Id of the associated charge |  
| trip_id                     | Unique Id of this trip | 
| tap_on_id                   | Tap ON ID provided | 
| tap_on_date                 | Date tapped on |  
| tap_off_id                  | Tap OFF ID provided (if variable) |  
| tap_off_date                | Date tapped off for variable fare |  
| original_amount             | Actual fare |  
| adjusted_amount             | Adjustment after capping was applied |  
| service_type                |  |  
| direction                   |  |  
| fare_type                   | flat, variable or retail|  
| trip_completion             | If trip was completed (if variable and no tap off, this is incomplete) |  

#### Adjustments
This table contains all the adjustments (including zero amount) that were done for a trip. An entry here means the a trip was qualified for a product.

| Field             | Description |
|:------------------|:--|
| product_id        | ID from the product table |  
| trip_id           | Trip ID from the trips table |  
| adjustment_id     | ID related to the group of adjustments. i.e.  |  
| adjustment_amount | Amount deducted from the actual fare |  
| adjustment_type   | Capping type from product table |  
| applied           | True if the fare was adjusted |  
| zone_ids_used     | Zones used in the adjustment |  

For the example earlier, it would have the following sample entries:

##### Product

| id | capping_type | rules_default_limit |
|:- |:-|:-|
| P1| DAILY | 5.0 |
| P2| WEEKLY | 40.0 |

##### Trip

| trip_id | tap_on_date | original_amount | adjusted_amount | trip_completion |
|:-|:-|:-|:-|:-|
| T1 | 2021-07-20 13:00:00.000 | 2.0  | 2.0  | complete  |
| T2 | 2021-07-20 14:00:00.000 | 2.0  | 2.0  | complete  |
| T3 | 2021-07-20 15:00:00.000 | 2.0  | 1.0  | complete  |
| T4 | 2021-07-21 13:00:00.000 | 3.0  | 3.0  | complete  |
| T5 | 2021-07-21 14:00:00.000 | 3.0  | 2.0  | complete  |
| T6 | 2021-07-21 15:00:00.000 | 3.0  | 0.0  | complete  |

###### Adjustments

| product_id| trip_id | adjustment_id | adjustment_amount | applied |
|:-|:-|:-|:-|:-|
| P1 | T1 | AJ1 | 0.0 | false |
| P1 | T2 | AJ1 | 0.0 | false |
| P1 | T3 | AJ1 | 1.0 | true  |
| P1 | T4 | AJ2 | 0.0 | false |
| P1 | T5 | AJ2 | 1.0 | true  |
| P1 | T6 | AJ2 | 3.0 | true  |
| P2 | T1 | AJ3 | 0.0 | false |
| P2 | T2 | AJ3 | 0.0 | false |
| P2 | T3 | AJ3 | 0.0 | false |
| P2 | T4 | AJ4 | 0.0 | false |
| P2 | T5 | AJ4 | 0.0 | false |
| P2 | T6 | AJ4 | 0.0 | false |


## Problem

1. Visualise the growth of the transactions volume (number) and amount which shows before and after the product was applied
2. (Optional) Provide insights about the visualisation. Were the capping product effective? If so, how effective? etc

### Note
You can use any library you like. Just declare it.

In [ ]:
# Imports
# NB. You dont have to use pandas.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [8]:
products_df = pd.read_csv('products-large.csv', sep=',')
trips_df = pd.read_csv('trips-large.csv', sep=',')
adjustments_df = pd.read_csv('adjustments-large.csv', sep=',')

In [1]:
import pandas as pd

In [4]:
products_df = pd.read_csv('products.csv', sep=',')
products_df

,id,product_type,capping_type,created_date,start_date_utc,status,rules_default_limit,rules_max_fare_value,rules_transaction_types,rules_from_time_local,rules_to_time_local,rules_weekly_cap_start_day,rules_capping_duration_in_minutes,rules_number_of_transfer
0,734f80f1-367a-4642-a482-ab18ae3668aa,CAPPING,DAILY_CAP,2019-07-17T15:30:56Z,2019-07-17T15:30:56Z,ACTIVE,5.2,5.2,"autoscan, variable",00:00:00,23:59:59,SUNDAY,0.0,NaN
1,37f8562e-969d-4d75-84dd-70709b9893c4,CAPPING,DAILY_CAP,2019-07-17T15:42:56Z,2019-07-17T15:42:56Z,ACTIVE,4.5,4.5,"autoscan, variable",18:00:00,03:59:59,SUNDAY,0.0,NaN
2,29edf630-870f-4b84-9113-ade5ab2413a0,CAPPING,DAILY_CAP,2019-07-17T15:44:57Z,2019-07-17T15:44:57Z,ACTIVE,5.2,5.2,"autoscan, variable",00:00:00,23:59:59,SUNDAY,0.0,NaN
3,9e564b22-fb50-4e66-b947-52fb50bacbc0,CAPPING,DAILY_CAP,2019-07-17T15:54:18Z,2019-07-17T15:54:18Z,ACTIVE,5.2,5.2,"autoscan, variable",00:00:00,23:59:59,SUNDAY,0.0,NaN
4,41791521-18f3-45c3-9c40-e611ac0cdade,CAPPING,DAILY_CAP,2019-07-17T15:56:29Z,2019-07-17T15:56:29Z,ACTIVE,5.2,5.2,"autoscan, variable",00:00:00,23:59:59,SUNDAY,0.0,NaN
5,b6de8022-2d7b-46db-aa9f-5bd5e77185b9,CAPPING,DAILY_CAP,2019-07-17T15:57:54Z,2019-07-17T15:57:54Z,ACTIVE,7.8,7.8,"autoscan, variable",00:00:00,23:59:59,SUNDAY,0.0,NaN


In [5]:
trips_df = pd.read_csv('trips.csv', sep=',')
trips_df

,customer_id,charge_id,trip_id,tap_on_id,tap_on_date,tap_off_id,tap_off_date,original_amount,adjusted_amount,service_type,direction,fare_type,trip_completion
0,cce09b29-9679-4442-8608-e3ebd6d4b6a7,955ebe41-8ebc-431d-b47e-1de03a9000b1,955ebe41-8ebc-431d-b47e-1de03a9000b1,294FCABF06B763DFEF9D07EBE036B9E3FF55BC33,2019-08-09 14:02:58.000,NaN,NaN,3.00,3.00,bus,inbound,retail_fare,complete
1,cce09b29-9679-4442-8608-e3ebd6d4b6a7,704fa471-9b8a-49e3-820f-23c952b70634,704fa471-9b8a-49e3-820f-23c952b70634,DD11C3C6DA5B315F9D0CB9419971BAFD1F739160,2019-06-16 05:52:01.000,NaN,NaN,1.50,1.50,bus,outbound,retail_fare,complete
2,cce09b29-9679-4442-8608-e3ebd6d4b6a7,b215446f-d3f6-430a-bfcf-2b85e6ab967c,81e6196d-593f-4880-81e3-b00c30e144cd,436460362CC9FE827C422BF31A6248874B3EE2D6,2019-08-14 14:41:57.000,NaN,NaN,2.50,2.50,bus,inbound,retail_fare,complete
3,72c4422d-4c9f-4484-8f8e-be57d03bd053,82adc4d6-efef-467a-a9cb-12c36f9cf140,82adc4d6-efef-467a-a9cb-12c36f9cf140,18E637167E49C0DCD012D24B163D471CEBB3AB7F,2019-06-14 16:01:01.000,NaN,NaN,2.50,2.50,bus,outbound,retail_fare,complete
4,4178c0f2-cc5f-408e-b013-be6a299ce019,1386d924-6242-4737-b1eb-dbf613f2ed0e,1386d924-6242-4737-b1eb-dbf613f2ed0e,1B8C25294DBCFF462AE0D046AA51EB0FF48ABEDA,2019-06-15 12:35:57.000,NaN,NaN,1.70,1.70,bus,outbound,retail_fare,complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425420,3f226c4b-766d-49d4-981d-ff0e3dfe376f,f2694ff3-d0b6-4013-8624-12063a11386e,f2694ff3-d0b6-4013-8624-12063a11386e,486931B6799C61D737A8C17004A8B93635F4CEE6,2019-06-20 21:33:37.000,NaN,NaN,2.50,2.50,bus,inbound,retail_fare,complete
425421,0937fc80-a9d7-4336-8fd7-604c389a4117,d96097eb-5950-4ec5-8e02-c4cbcc662c48,d96097eb-5950-4ec5-8e02-c4cbcc662c48,0DC1B37307C2BC61D4570559D4BA591A3E0385A4,2019-08-16 15:40:11.000,NaN,NaN,1.25,1.25,bus,inbound,retail_fare,complete
425422,7e6b358a-ddfc-4327-87bc-70f47b19fb5f,859f0aaf-908c-48ae-a390-4b5347ef9d63,859f0aaf-908c-48ae-a390-4b5347ef9d63,857824153A89B23A7739F3E5F79EBD6F61BBB684,2019-09-19 17:13:04.000,NaN,NaN,1.70,1.70,bus,inbound,retail_fare,complete
425423,49e88900-ca97-4593-bacb-3d6ebab70667,0fde3d85-8d27-4654-996a-6e733035a890,0fde3d85-8d27-4654-996a-6e733035a890,FEFEF79699D99E2689185A0F0D586FD58987B9E9,2019-07-05 10:28:02.000,NaN,NaN,5.00,5.00,bus,inbound,retail_fare,complete


In [ ]:
adjustments_df = pd.read_csv('adjustments.csv.gz', sep=',', compression='infer')
adjustments_df

,product_id,trip_id,adjustment_id,adjustment_amount,adjustment_type,applied,zone_ids_used
0,b6de8022-2d7b-46db-aa9f-5bd5e77185b9,3cc69047-1cb5-492f-a198-07d9abf12768,66c833c1-e424-4f3c-932d-2532ff6f2be9,0.0,DAILY_CAP,False,NaN
1,734f80f1-367a-4642-a482-ab18ae3668aa,2da54454-c75e-40df-94be-d2e3193482fc,21bc9dc5-7ca2-49c7-bd84-aaefd434d4d8,0.0,DAILY_CAP,False,NaN
2,b6de8022-2d7b-46db-aa9f-5bd5e77185b9,e72fea60-4a9e-450c-b2ab-fcf9e4811a19,89c39cd0-95b8-4d39-9245-3a4a4df21f2b,0.0,DAILY_CAP,False,NaN
3,37f8562e-969d-4d75-84dd-70709b9893c4,fc698a2e-47c5-471e-afb2-30766da7cc41,e0100722-e09a-4e3d-bd14-e78484ff8aff,0.0,DAILY_CAP,False,NaN
4,734f80f1-367a-4642-a482-ab18ae3668aa,c0917260-9876-4e7c-adbb-3396defcf518,3270ad3e-5735-4c4e-acdb-78fa6e2aa92a,0.0,DAILY_CAP,False,NaN
...,...,...,...,...,...,...,...
46095,734f80f1-367a-4642-a482-ab18ae3668aa,cdd1d656-3697-4ba0-9e09-bbea09cbd649,9b35d6a6-8641-4545-933b-d8a2b3c2b2a6,0.0,DAILY_CAP,False,NaN
46096,734f80f1-367a-4642-a482-ab18ae3668aa,d54ec491-ba2f-4888-b43e-fc96a55bac7d,2cdf59f1-4b16-423b-8944-d0175331d6ce,0.0,DAILY_CAP,False,NaN
46097,b6de8022-2d7b-46db-aa9f-5bd5e77185b9,f8249954-9338-4c5f-bcdf-c3ad6b23b59d,b2c3a587-3b9f-40e0-aaa3-8a73a8baf966,0.0,DAILY_CAP,False,NaN
46098,734f80f1-367a-4642-a482-ab18ae3668aa,02e9d305-61b0-4caf-b673-a653058d94e2,7433a275-d417-4448-ab4d-ac7f965dcae4,0.0,DAILY_CAP,False,NaN
